# PDF Renamer
This is part of a small set of tools I've developed for extracting data and categorizing/indexing test PDFs for Mu Alpha Theta


In [5]:
!pip install -U -q "google-genai"
!pip install -U -q "PyPDF2"
!pip install -U -q "pdfplumber"
!pip install -U -q "dotenv"


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [7]:
import requests
import pathlib
from google import genai
from PyPDF2 import PdfReader, PdfWriter
import time
import os
import json
import math
import csv
import re
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key = api_key)

In [11]:
file_names = ['Calc Mar 2022 Reg Indiv Test.pdf','Calc Mar 2022 Reg Indiv Ans and Sols.pdf','Stats Mar 2022 Reg Indiv Ans and Sols.pdf','Stats Mar 2022 Reg Indiv Test.pdf','Prec Mar 2022 Reg Indiv Test.pdf','Prec Mar 2022 Reg Indiv Ans and Sols.pdf','Geo Mar 2022 Reg Indiv Ans and Sols.pdf','Geo Mar 2022 Reg Indiv Test.pdf','Alg 2 Mar 2022 Reg Indiv Ans and Sols.pdf','Alg 2 Mar 2022 Reg Indiv Test.pdf','Alg 1 Mar 2022 Reg Indiv Ans and Sols.pdf','Alg 1 Mar 2022 Reg Indiv Test.pdf','Geometry Individual.pdf','Geometry Individual Solutions.pdf','Calc Indiv.pdf','Calc Indiv Solutions.pdf','Precalc indiv.pdf','Precalc indiv solutions.pdf','Algebra 2 Indiv.pdf','Algebra 2 Indiv Solutions.pdf','Algebra 1 Indiv.pdf','Algebra 1 Indiv Solutions.pdf','Stats Jan 2022 Buchholz SW Indiv Ans and Sols.pdf','Stats Jan 2022 Buchholz SW Indiv Test.pdf','Alg 2 Feb 2022 AHSPB SW Indiv Test.pdf','Alg 2 Feb 2022 AHSPB SW Indiv Ans and Sols.pdf','Calc Feb 2022 AHSPB SW Indiv Ans & Sols.pdf','Calc Feb 2022 AHSPB SW Indiv Test.pdf','Prec Feb 2022 AHSPB SW Indiv Ans & Sols.pdf','Prec Feb 2022 AHSPB SW Indiv Test.pdf','Stats Feb 2022 AHSPB SW Indiv Ans and Sols.pdf','Stats Feb 2022 AHSPB SW Indiv Test.pdf','Geo Feb 2022 AHSPB SW Indiv Test.pdf','Geo Feb 2022 AHSPB SW Indiv Ans & Sols.pdf','Alg 1 Feb 2022 AHSPB SW Indiv Ans & Sols.pdf','Alg 1 Feb 2022 AHSPB SW Indiv Test.pdf','Stats Feb 2022 Reg Indiv Ans and Sols.pdf','Stats Feb 2022 Reg Indiv Test.pdf','Prec Feb 2022 Reg Indiv Test.pdf','Prec Feb 2022 Reg Indiv Ans and Sols.pdf','Geo Feb 2022 Reg Indiv Test.pdf','Geo Feb 2022 Reg Indiv Ans and Sols.pdf','Calc Feb 2022 Reg Indiv Ans and Sols.pdf','Calc Feb 2022 Reg Indiv Test.pdf','Alg 2 Feb 2022 Reg Indiv Ans and Sols.pdf','Alg 2 Feb 2022 Reg Indiv Test.pdf','Alg 1 Feb 2022 Reg Indiv Test Ans and Sols.pdf','Alg 1 Feb 2022 Reg Indiv Test.pdf','Alg 2 Jan 2022 Reg Indiv Ans and Sols.pdf','Alg 2 Jan 2022 Reg Indiv Test.pdf','Calc Jan 2022 Reg Indiv Ans and Sols.pdf','Calc Jan 2022 Reg Indiv Test.pdf','Geo Jan 2022 Reg Indiv Ans and Sols.pdf','Geo Jan 2022 Reg Indiv Test.pdf','Prec Jan 2022 Reg Indiv Test.pdf','Prec Jan 2022 Reg Indiv Ans and Sols.pdf','Stats Jan 2022 Reg Indiv Test.pdf','Stats Jan 2022 Reg Indiv Ans and Sols.pdf','Alg 1 Jan 2022 Reg Indiv Ans and Sols.pdf','Alg 1 Jan 2022 Reg Indiv Test.pdf','Alg 2 Mar 2025 NSU SW Indiv Ans and Sols.pdf','Alg 1 Mar 2025 NSU SW Indiv Ans and Sols.pdf','Prec Mar 2025 NSU SW Indiv Test.pdf','Stats Mar 2025 NSU SW Indiv Ans and Sols.pdf','Stats Mar 2025 NSU SW Indiv Test.pdf','Alg 1 Mar 2025 NSU SW Indiv Test.pdf','Alg 2 Mar 2025 NSU SW Indiv Test.pdf','Prec Mar 2025 NSU SW Indiv Ans and Sols.pdf','Geo Mar 2025 Reg Indiv Ans and Sols.pdf','Geo Mar 2025 Reg Indiv Test.pdf','Prec Mar 2025 Reg Indiv Test.pdf','Prec Mar 2025 Reg Indiv Ans and Sols.pdf','Stats Mar 2025 Reg Indiv Test.pdf','Stats Mar 2025 Reg Indiv Ans and Sols.pdf','Alg 1 Mar 2025 Reg Indiv Ans and Sols.pdf','Alg 1 Mar 2025 Reg Indiv Test.pdf','Calc Mar 2025 Reg Indiv Test.pdf','Calc Mar 2025 Reg Indiv Ans and Sols.pdf','Alg 2 Mar 2025 Reg Indiv Test.pdf','Alg 2 Mar 2025 Reg Indiv Ans and Sols.pdf','Alg 1 Feb 2025 Reg Indiv Test.pdf','Alg 1 Feb 2025 Reg Indiv Ans and Sols.pdf','Stats Feb 2025 Reg Indiv Ans and Sols.pdf','Geo Feb 2025 Reg Indiv Test.pdf','Geo Feb 2025 Reg Indiv Ans and Sols.pdf','Alg 2 Feb 2025 Reg Indiv Ans and Sols.pdf','Stats Feb 2025 Reg Indiv Test.pdf','Alg 2 Feb 2025 Reg Indiv Test.pdf','Prec Feb 2025 Reg Indiv Ans and Sols.pdf','Prec Feb 2025 Reg Indiv Test.pdf','Calc Feb 2025 Reg Indiv Ans and Sols.pdf','Calc Feb 2025 Reg Indiv Test.pdf','Geo Feb 2025 KING SW Indiv Ans and Sols - With Corrections.pdf','Prec Feb 2025 KING SW Indiv Ans and Sols - With Corrections.pdf','Calc Feb 2025 KING SW Indiv Ans and Sols - With Corrections.pdf','Alg 2 Feb 2025 KING SW Indiv Ans and Sols - With Corrections.pdf','Alg 1 Feb 2025 KING SW Indiv Ans and Sols - With Corrections.pdf','Prec Feb 2025 KING SW Indiv Test.pdf','Calc Feb 2025 KING SW Indiv Test.pdf','Alg 2 Feb 2025 KING SW Indiv Test.pdf','Geo Feb 2025 KING SW Indiv Test.pdf','Alg 1 Feb 2025 KING SW Indiv Test.pdf','Stats Feb 2025 KING SW Indiv Ans and Sols - With Corrections.pdf','Stats Feb 2025 KING SW Indiv Test - With Corrections.pdf','KHS 2025 Precalc Indiv Feb SOL.pdf','Stats Feb 2025 KING SW Indiv Ans and Sols.pdf','2025 KHS Alg 1 Indiv Feb SOL.pdf','2025 KHS Geom Indiv Feb SOL.pdf','Stats Feb 2025 KING SW Indiv Test-combined_1.pdf','Prec Jan 2025 Reg Indiv Ans and Sols.pdf','Geo Jan 2025 Reg Indiv Ans and Sols.pdf','Alg 1 Jan 2025 Reg Indiv Ans and Sols.pdf','Alg 2 Jan 2025 Reg Indiv Test.pdf','Alg 2 Jan 2025 Reg Indiv Ans and Sols.pdf','Stats Jan 2025 Reg Indiv Ans and Sols.pdf','Stats Jan 2025 Reg Indiv Test.pdf','Calc Jan 2025 Reg Indiv Ans and Sols.pdf','Calc Jan 2025 Reg Indiv Test.pdf','Prec Jan 2025 Reg Indiv Test.pdf','Geo Jan 2025 Reg Indiv Test.pdf','Alg 1 Jan 2025 Reg Indiv Test.pdf','Full_Algebra 1_Indiv_Maclay Regional January 2025.pdf','Full_Precalculus_Indiv_Maclay Regional January 2025.pdf','Full_Statistics_Indiv_Maclay Regional January 2025.pdf','Full_Geometry_Indiv_Maclay Regional January 2025.pdf','Full_Algebra 2_Indiv_Maclay Regional January 2025.pdf','Full_Calculus_Indiv_Maclay Regional January 2025.pdf','Full_Statistics_Indiv_Orlando Science Regional January 2025.pdf','Full_Precalculus_Indiv_Orlando Science Regional January 2025.pdf','Full_Calculus_Indiv_Orlando Science Regional January 2025.pdf','Full_Geometry_Indiv_Orlando Science Regional January 2025.pdf','Full_Algebra 1_Indiv_Orlando Science Regional January 2025.pdf','Full_Algebra 2_Indiv_Orlando Science Regional January 2025.pdf','Full_Precalculus_Indiv_Palm Harbor Regional January 2025.pdf','Full_Statistics_Indiv_Palm Harbor Regional January 2025.pdf','Full_Algebra 2_Indiv_Palm Harbor Regional January 2025.pdf','Full_Geometry_Indiv_Palm Harbor Regional January 2025.pdf','Full_Calculus_Indiv_Palm Harbor Regional January 2025.pdf','Full_Algebra 1_Indiv_Palm Harbor Regional January 2025.pdf','Full_Statistics_Indiv_AHS Broward Region 5 January Regional 2025.pdf','Full_Precalculus_Indiv_AHS Broward Region 5 January Regional 2025.pdf','Full_Calculus_Indiv_AHS Broward Region 5 January Regional 2025.pdf','Full_Algebra 2_Indiv_AHS Broward Region 5 January Regional 2025.pdf','Full_Geometry_Indiv_AHS Broward Region 5 January Regional 2025.pdf','Full_Algebra 1_Indiv_AHS Broward Region 5 January Regional 2025.pdf','Algebra I Individual Test - Chiles Statewide 2025.pdf','Algebra I Individual Solutions - Chiles Statewide 2025.pdf','Algebra I Individual Answers - Chiles Statewide 2025.pdf','Algebra 2 Individual Answers and Solutions - Chiles Statewide 2025.pdf','Algebra 2 Individual Test - Chiles Statewide 2025.pdf','Tallahassee Jan Alg 2 Individual Test.pdf','Tallahassee Jan Alg 2 Indiv SOLUTIONS.pdf','Calculus Individual Test - Chiles Statewide 2025.pdf','Calculus Individual Answers and Solutions - Chiles Statewide 2025.pdf','Calc Jan 2025 Statewide Indiv Test.pdf','Calc Jan 2025 Statewide Indiv Ans and Sols.pdf','Precalculus Individual Test - Chiles Statewide 2025.pdf','Precalculus Individual Solutions - Chiles Statewide 2025.pdf','Precalculus Individual Answers - Chiles Statewide 2025.pdf','Stats Jan 2025 Chiles SW Indiv Ans and Sols.pdf','Stats Jan 2025 Chiles SW Indiv Test.pdf','Stats Jan 2025 Chiles SW Indiv Ans and Sols.pdf','Stats Jan 2025 Chiles SW Indiv Test.pdf','Geo Jan 2025 Statewide Indiv Ans and Sols.pdf','Geo Jan 2025 Statewide Indiv Test.pdf','Geo Jan 2025 Statewide Indiv Ans and Sols.pdf','Geo Jan 2025 Statewide Indiv Test.pdf','Nunn Dec 2024 Indiv Ans and Sols.pdf','Nunn Dec 2024 Indiv Test.pdf','Dostal Dec 2024 Indiv Test.pdf','Dostal Dec 2024 Indiv Ans and Sols.pdf','Hiller Dec 2024 Indiv Ans and Sols.pdf','Hiller Dec 2024 Indiv Test.pdf','Full_Nunn_Indiv_HDN 2024.pdf','Full_Hiller_Indiv_HDN 2024.pdf','Full_Dostal_Indiv_HDN 2024.pdf','Alg 2 Mar 2024 Reg Indiv Test.pdf','Alg 2 Mar 2024 Reg Indiv Ans and Sols.pdf','Stats Mar 2024 Reg Indiv Test.pdf','Stats Mar 2024 Reg Indiv Ans and Sols.pdf','2024 March Errata Sheet Precalc Indiv (ver2).pdf','Prec Mar 2024 Reg Indiv Ans and Sols.pdf','Prec Mar 2024 Reg Indiv Test.pdf','Geo Mar 2024 Reg Indiv Test.pdf','Geo Mar 2024 Reg Indiv Ans and Sols.pdf','Calc Mar 2024 Reg Indiv Test.pdf','Calc Mar 2024 Reg Indiv Ans and Sols.pdf','Alg 1 Mar 2024 Reg Indiv Test.pdf','Alg 1 Mar 2024 Reg Indiv Ans and Sols.pdf','Alg 1 Feb 2024 Reg Indiv Ans and Sols.pdf','Stats Feb 2024 Reg Indiv Ans and Sols.pdf','Stats Feb 2024 Reg Indiv Test.pdf','Calc Feb 2024 Reg Indiv Test.pdf','Calc Feb 2024 Reg Indiv Ans and Sols.pdf','Prec Feb 2024 Reg Indiv Ans and Sols.pdf','Prec Feb 2024 Reg Indiv Test.pdf','Geo Feb 2024 Reg Indiv Test.pdf','Geo Feb 2024 Reg Indiv Ans and Sols.pdf','Alg 2 Feb 2024 Reg Indiv Test.pdf','Alg 2 Feb 2024 Reg Indiv Ans and Sols.pdf','Alg 1 Feb 2024 Reg Indiv Test.pdf','Alg 2 Jan 2024 Reg Indiv Ans and Sols.pdf','Prec Jan 2024 Reg Indiv Ans and Sols.pdf','Prec Jan 2024 Reg Indiv Test.pdf','Geo Jan 2024 Reg Indiv Ans and Sols.pdf','Geo Jan 2024 Reg Indiv Test.pdf','Stats Jan 2024 Reg Indiv Test.pdf','Stats Jan 2024 Reg Indiv Ans and Sols.pdf','Calc Jan 2024 Reg Indiv Test.pdf','Calc Jan 2024 Reg Indiv Ans and Sols.pdf','Alg 2 Jan 2024 Reg Indiv Test.pdf','Alg 1 Jan 2024 Reg Indiv Test.pdf','Alg 1 Jan 2024 Reg Indiv Ans and Sols.pdf','Dostal Dec 2023 Indiv Test.pdf','Dostal Dec 2023 Indiv Ans and Sols.pdf','Hiller Dec 2023 Indiv Solutions.pdf','Hiller Dec 2023 Indiv Test.pdf','Nunn Dec 2023 Indiv Test.pdf','Nunn Dec 2023 Indiv Ans and Sols.pdf','Alg 2 Jan 1324 BC SW Indiv Ans and Sols.pdf','Alg 1 Jan 1324 BC SW Indiv Ans and Sols With Errata.pdf','Geo Jan 1324 BC SW Indiv Ans and Sols With Errata.pdf','Stats Jan 1324 BC SW Indiv Test.pdf','Stats Jan 1324 BC SW Indiv Ans and Sols.pdf','Geo Jan 1324 BC SW Indiv Test.pdf','Alg 1 Jan 1324 BC SW Indiv Test.pdf','Alg 2 Jan 1324 BC SW Indiv Test.pdf','Alg 1 Mar 2024 NSU SW Indiv Test.pdf','Alg 1 Mar 2024 NSU SW Indiv Ans and Sols.pdf','Geo Mar 2024 NSU SW Indiv Ans and Sols.pdf','Geo Mar 2024 NSU SW Indiv Test.pdf','Alg 2 Mar 2024 NSU SW Team Indiv Test with Ans and Sols and Errata.pdf','Calc Mar 2024 NSU SW Indiv Ans and Sols.pdf','Calc Mar 2024 NSU SW Indiv Test V3.pdf','Stats Mar 2024 NSU SW Indiv Ans and Sols.pdf','Stats Mar 2024 NSU SW Indiv Test.pdf','Calc Feb 2024 KING SW Indiv Ans and Sols with ERRATA.pdf','Calc Feb 2024 KING SW Indiv Test.pdf','Alg 2 Feb 2024 KING SW Indiv Ans and Sols with ERRATA.pdf','Alg 2 Feb 2024 KING SW Indiv Test.pdf','Prec Feb 2024 KING SW Indiv Ans and Sols with ERRATA.pdf','Prec Feb 2024 KING SW Indiv Test.pdf','Geo Feb 2024 KING SW Indiv Test.pdf','Geo Feb 2024 KING SW Indiv Ans and Sols.pdf','Alg 1 Feb 2024 KING SW Indiv Ans and Sols with ERRATA.pdf','Alg 1 Feb 2024 KING SW Indiv Test.pdf','Stats Feb 2024 KING SW Indiv Ans and Sols.pdf','Stats Feb 2024 KING SW Indiv Test.pdf','Alpha Indiv Solutions.pdf','Alpha Indiv.pdf','March 2023 Theta Individual Solutions.pdf','March 2023 Theta Individual Test.pdf','Stats Mar 2023 Reg Indiv Test.pdf','Stats Mar 2023 Reg Indiv Ans and Sols.pdf','Calc Mar 2023 Reg Indiv Test.pdf','Calc Mar 2023 Reg Indiv Ans and Sols.pdf','Geo Mar 2023 Reg Indiv Test.pdf','Geo Mar 2023 Reg Indiv Ans and Sols.pdf','Alg 2 Mar 2023 Reg Indiv Test.pdf','Alg 2 Mar 2023 Reg Indiv Ans and Sols.pdf','Prec Mar 2023 Reg Indiv Test.pdf','Prec Mar 2023 Reg Indiv Ans and Sols.pdf','Alg 1 Mar 2023 Reg Indiv Test.pdf','Alg 1 Mar 2023 Reg Indiv Ans and Sols.pdf','Stats Feb 2023 NSU SW Indiv Ans and Sols.pdf','StatisticsIndividualTestFinal.pdf','PrecalculusIndividualTestFinal.pdf','Precal_ Feb 2023 NSU SW Indiv Test Solutions Final Review 1 K C.pdf','GeometryIndividualTestFinal.pdf','Geo Feb 2023 NSU SW Indiv Ans and Sols.pdf','CalculusIndividualTestFinal.pdf','Calc_Feb 2023 NSU SW Indiv Test Sols.pdf','Alg 2 Feb 2023 NSU SW Indiv Ans and Sols.pdf','Algebra2IndividualTestFinal.pdf','Alg 1 Feb 2023 NSU SW Indiv Ans and Sols.pdf','Algebra1IndividualTestFinal.pdf','Calc Feb 2023 Reg Indiv Test.pdf','Calc Feb 2023 Reg Indiv Ans and Sols.pdf','Prec Feb 2023 Reg Indiv Test.pdf','Prec Feb 2023 Reg Indiv Ans and Sols.pdf','Alg 2 Feb 2023 Reg Indiv Test.pdf','Alg 2 Feb 2023 Reg Indiv Ans and Sols.pdf','Stats Feb 2023 Reg Indiv Test.pdf','Stats Feb 2023 Reg Indiv Ans and Sols.pdf','Alg 1 Feb 2023 Reg Indiv Test.pdf','Alg 1 Feb 2023 Reg Indiv Ans and Sols.pdf','Geo Feb 2023 Reg Indiv Test.pdf','Geo Feb 2023 Reg Indiv Ans and Sols.pdf','Stats Jan 2023 Reg Indiv Test.pdf','Stats Jan 2023 Reg Indiv Ans and Sols.pdf','Calc Jan 2023 Reg Indiv Test.pdf','Calc Jan 2023 Reg Indiv Ans and Sols.pdf','Alg 1 Jan 2023 Reg Indiv Ans and Sols.pdf','Alg 1 Jan 2023 Reg Indiv Test.pdf','Alg 2 Jan 2023 Reg Indiv Test.pdf','Alg 2 Jan 2023 Reg Indiv Ans and Sols.pdf','Geo Jan 2023 Reg Indiv Ans and Sols.pdf','Geo Jan 2023 Reg Indiv Test.pdf','Prec Jan 2023 Reg Indiv Test.pdf','Prec Jan 2023 Reg Indiv Ans and Sols.pdf','Nunn Dec 2022 Indiv Test.pdf','Nunn Dec 2022 Indiv Ans and Sols.pdf','Hiller Dec 2022 Indiv Test.pdf','Hiller Dec 2022 Indiv Ans and Sols.pdf','Dostal Dec 2022 Indiv Test.pdf','Dostal Dec 2022 Indiv Ans and Sols.pdf','2025 Nationals Geometry Individual.pdf','2025 Nationals Geometry Individual Solutions.pdf','2025 Nationals Theta Individual.pdf','2025 Nationals Theta Individual Solutions.pdf','Nats 2025 Stats Indiv Test MRG-BR.pdf','Nats 2025 Stats Indiv Ans and Sols MRG-BR.pdf','2025 Nationals Mu Individual Solutions.pdf','2025 Nationals Mu Individual.pdf','Errata - Individual.pdf','Alpha Individual MAO Nationals 2025 - Solutions.pdf','Alpha Individual MAO Nationals 2025.pdf','2025 States Calculus Individual Solutions.pdf','2025 States Calculus Individual.pdf','2025 States Alpha Individual.pdf','2025 States Alpha Individual Solutions.pdf','Theta Individual FAMAT States 2025.pdf','Theta Individual FAMAT States 2025 - Solutions.pdf','States 2025 Stats Unplugged Indiv Test.pdf','2025 FAMAT State Convention Statistics Individual.pdf','2025 FAMAT State Convention Statistics Individual Answers.pdf','2024 States Alpha Individual.pdf','2024 States Alpha Individual Solutions.pdf','States 2024 Stats Indiv Ans and Sols.pdf','States 2024 Stats Indiv Test.pdf','2024 States Calculus Individual Solutions.pdf','2024 States Calculus Individual.pdf','2024 States Theta Individual Solutions.pdf','2024 States Theta Individual.pdf','2023 States Alpha Individual Solutions.pdf','2023 States Alpha Individual.pdf','2023 Theta Individual.pdf','2023 Theta Individual Answers and Solutions.pdf','States 2023 Calculus Individual Solutions.pdf','States 2023 Calculus Individual Test.pdf','2022 Theta Individual Solutions.pdf','2022 Theta Individual.pdf','2022 States Calculus Individual - Test.pdf','2022 States Calculus Individual - Solutions.pdf','2022 States Alpha Individual Solutions.pdf','2022 States Alpha Individual.pdf','theta individual state 2018.pdf','2018 States Calc Individual Solutions - Final.pdf','2018 States Calc Individual Test - Final.pdf','Alpha Individual State 2018.pdf','theta individual state 2017.pdf','Theta individual state 2017 sol.pdf','2017 States Calculus Individual - Test Only.pdf','2017 States Calculus Individual - Solutions.pdf','Alpha Indiv Part 1 Solutions.pdf','Alpha Indiv Part 2.pdf','Alpha Indiv Part 2 Solutions.pdf','Alpha Indiv Part 1.pdf','Theta Individual State 2019.pdf','Theta Individual State 2019 solutions.pdf','Mu Individual.pdf','Mu Individual Solutions.pdf','Alpha_Indiv_States_2019_final.pdf','Alpha_Indiv_States_2019_final-sols.pdf','States 2021 Theta Indiv Solutions.pdf','States 2021 Theta Indiv.pdf','T_Calculus_Individual_States_2021.pdf','S_Calculus_Individual_States_2021.pdf','S_Alpha_Individual_States_2021.pdf','T_Alpha_Individual_States_2021.pdf','Theta Individual - States 2016 - Test Only.pdf','Calculus Individual - States 2016 - Test Only.pdf','Alpha Individual - States 2016 - Test Only.pdf']


name_blocks = []

output_csv = "file_names_output.csv"

def chunk (input_array, step_size):
  for i in range(0, len(input_array), step_size):
    yield input_array[i:(i + step_size)]

for group in chunk(file_names, 50):
  try:
    response = client.models.generate_content(
                model='gemini-2.0-flash',
                contents=[
                    """You are a file renaming assistant. You will be given an array of file names from a math competition archive. Each file name may be inconsistently formatted.

                    Your job is to **standardize** them using the format:

                        {Year} + " " + {Test date} + " " + {Event Type} + " " + {Division} + " " + {Test type}

                    Where:
                    - Test date or level: Jan, Feb, March, States, or Nationals
                    - Even type: Regional, Statewide, National, States
                    - Division: Algebra 2, Precalc, or Calc (even if written as ALG2, prec, calc1, etc.)
                    - Test type: Indiv, Team, Ciphering, Team Round, etc.

                    The parts might be out of order or abbreviated. Do your best to **reorder, correct**, and **fill in** based on common sense and examples.


                    Use title case for each component (e.g. "Precalc", not "precalc").
                    Separate components with `" "` (space).

                    Return a **JSON array of objects**, where each object contains:
                    ```json
                    {
                      "original": "<original file name>",
                      "renamed": "<standardized file name>"
                    }
                    Here are the file names:""" + f"{group}"
                ]
            )
    response_text = getattr(response, "text", None) or getattr(response.candidates[0], "content", "")
    match = re.search(r"```json\s*(.*?)\s*```", response_text, re.DOTALL)
    clean_json = match.group(1).strip() if match else response_text.strip()
    name_blocks.append(clean_json)

    print(f" Successfully processed batch")


    time.sleep(.5)

  except Exception as e:
      print(f" Error processing batch")


all_questions = []
for idx, block in enumerate(name_blocks):
    try:
        parsed = json.loads(block)

        if not isinstance(parsed, list):
            print(f" Skipping block {idx+1} (not a list)")
            continue

        for q in parsed:
            if not isinstance(q, dict):
                print(f" Skipping malformed question in block {idx+1}: {q}")
                continue

            question_text = q.get("original", "").strip()
            choices_raw = q.get("renamed", {})

            all_questions.append([question_text, choices_raw])

    except json.JSONDecodeError:
        print(f"JSON decode error in block {idx+1}: {block[:100]}")

# Save results to CSV
with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["question_text", "answer_choices"])
    writer.writerows(all_questions)

print(f"\n Finished! Saved {len(all_questions)} questions to {output_csv}")

 Successfully processed batch
 Successfully processed batch
 Successfully processed batch
 Successfully processed batch
 Successfully processed batch
 Successfully processed batch
 Successfully processed batch
 Successfully processed batch

 Finished! Saved 373 questions to file_names_output.csv
